In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from sklearn import model_selection
from IPython.display import display, HTML
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph
from stellargraph import datasets
import networkx as nx
from stellargraph.data import BiasedRandomWalk
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os
import random
import stellargraph as sg
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [2]:
LIMIT = 250

In [3]:
bitcoin_address =  pd.read_csv('bitcoin_address.csv',nrows=LIMIT)

bitcoin_address_link =  pd.read_csv('bitcoin_address_link.csv',nrows=LIMIT)

page =  pd.read_csv('page.csv',nrows=LIMIT)

page_link =  pd.read_csv('page_link.csv',nrows=LIMIT)

domain =  pd.read_csv('domain.csv',nrows=LIMIT)

In [4]:
G1 = nx.Graph()
node_subjects = {}

#insert bitcoin address nodes

for index, row in tqdm(bitcoin_address.iterrows()):
    G1.add_node(int(row.id))
    node_subjects[row.id]=1
    
#insert pages

for index, row in tqdm(page.iterrows()):
    G1.add_node(int(row.id))
    node_subjects[row.id]=2
    
#insert page - bitcoin addres edges

for index, row in tqdm(bitcoin_address_link.iterrows()):
    G1.add_edge(int(row.bitcoin_address), int(row.page))
    
#insert page - page edges

for index, row in tqdm(page_link.iterrows()):
    G1.add_edge(int(row.link_from), int(row.link_to ))
    
    
#insert domain

for index, row in tqdm(domain.iterrows()):
    G1.add_node(int(row.id))
    node_subjects[row.id]=3
    
#insert page - domain edges

for index, row in tqdm(page.iterrows()):
    G1.add_edge(int(row.id), int(row.domain))


In [37]:
g1 =  StellarGraph.from_networkx(G1)
print(g1.info())

StellarGraph: Undirected multigraph
 Nodes: 1055, Edges: 733

 Node types:
  default: [1055]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [733]
        Weights: all 1 (default)
        Features: none


In [38]:
edge_splitter_test = EdgeSplitter(g1)

graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(p=0.1, method="global")

print(graph_test.info())

** Sampled 73 positive and 73 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 1055, Edges: 660

 Node types:
  default: [1055]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [660]
        Weights: all 1 (default)
        Features: none


In [39]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, g1)
graph_train, examples, labels = edge_splitter_train.train_test_split(p=0.1, method="global")

(examples_train,examples_model_selection,labels_train,labels_model_selection,) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

** Sampled 66 positive and 66 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 1055, Edges: 594

 Node types:
  default: [1055]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [594]
        Weights: all 1 (default)
        Features: none


In [40]:
pd.DataFrame({
    'Split':['Training Set','Model Selection','Test set'],
    'Number of Examples':[len(examples_train),len(examples_model_selection),len(examples_test)]
})

,Split,Number of Examples
0,Training Set,99
1,Model Selection,33
2,Test set,146


In [10]:
import multiprocessing
P = 1.0
Q = 1.0
DIMENSIONS = 128
NUMOFWALKS =10
WALKOFLENGTH = 80
WINDOWSIZE =10
NUMITER = 1
WORKERS = multiprocessing.cpu_count()

In [11]:
from gensim.models import Word2Vec


def node2vec_embedding(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=NUMOFWALKS, length=WALKOFLENGTH, p=P, q=Q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=DIMENSIONS,
        window=WINDOWSIZE,
        min_count=0,
        sg=1,
        workers=WORKERS,
        iter=NUMITER,
    )

    get_embedding = lambda u: model.wv[u]

    return get_embedding

In [12]:
embedding_train = node2vec_embedding(graph_train, "Train Graph")

Number of random walks for 'Train Graph': 24850


In [13]:
def linkExamplesToFeatures(linkExamples, transformNode, binaryOperator):
    return [binaryOperator(transformNode(src), transformNode(dst)) for src, dst in linkExamples]

In [14]:
def trainLinkPredictionModel(linkExamples, linkLabels, getEmbedding, binaryOperator):
    classif = linkPredictionClassifier()
    linkFeatures = linkExamplesToFeatures(linkExamples, getEmbedding, binaryOperator)
    classif.fit(linkFeatures, linkLabels)
    return classif

In [15]:
def linkPredictionClassifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])

In [25]:
def evaluateLinkPredictionModel(clf, link_examples_test, link_labels_test, get_embedding, binary_operator):
    link_features_test = linkExamplesToFeatures(link_examples_test, get_embedding, binary_operator)
    score = evaluate_ROC_AUC(clf, link_features_test, link_labels_test)
    return score

In [26]:
def evaluate_ROC_AUC(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [27]:
def operator_hadamard(u, v):
    return u * v

In [28]:
def operator_l1(u, v):
    return np.abs(u - v)

In [29]:
def operator_l2(u, v):
    return (u - v) ** 2

In [30]:
def operator_avg(u, v):
    return (u + v) / 2.0

In [31]:
def run_link_prediction(binary_operator):
    clf = trainLinkPredictionModel(examples_train, labels_train, embedding_train, binary_operator)
    
    score = evaluateLinkPredictionModel(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }

In [32]:
binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [33]:
results = [run_link_prediction(op) for op in binary_operators]
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

Best result from 'operator_l2'


,ROC AUC score
name,
operator_hadamard,0.862039
operator_l1,0.899817
operator_l2,0.901352
operator_avg,0.567190


In [34]:
embedding_test = node2vec_embedding(graph_test, "Test Graph")

Number of random walks for 'Test Graph': 24850


In [36]:
test_score = evaluateLinkPredictionModel(best_result["classifier"],examples_test,labels_test,embedding_test,best_result["binary_operator"],)
print(f"ROC AUC score on test set using '{best_result['binary_operator'].__name__}': {test_score}")

ROC AUC score on test set using 'operator_l2': 0.9000147928994083


In [ ]:
# print('LogisticRegressionCV',accuracy_score(y_test,y_pred),
#       ' precision : ',precision_score(y_test,y_pred,average='micro'),
#       ' recall : ',recall_score(y_test,y_pred,average='micro'),'\n')
# print(classification_report(y_test,y_pred))

In [ ]:
# plt.title("Confusion Matrix")
# sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,cmap="Blues",fmt="g",cbar=False)
# plt.show()